In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pennylane import numpy as pnp
import pennylane as qml
from pennylane.optimize import AdamOptimizer

# Define file paths
path_train = '~/Downloads/QARS/al5083/train/train.json'
path_test = '~/Downloads/QARS/al5083/test/test.json'

# Function to load JSON data
def load_json_to_df(path):
    try:
        with open(path, 'r') as file:
            data = json.load(file)
        return pd.json_normalize(data)
    except Exception as e:
        print(f"Error loading JSON file {path}: {e}")
        return None

# Load data
train_df = load_json_to_df(path_train)
test_df = load_json_to_df(path_test)

# Check and print column names
print("Training DataFrame columns:")
print(train_df.columns)
print("Test DataFrame columns:")
print(test_df.columns)

# Handle missing values
train_df = train_df.ffill()
test_df = test_df.ffill()

# Verify column names and target
print(train_df.head())
print(test_df.head())

# Replace 'target' with the actual name of your target column
target_column = 'target'  # Update this with the correct target column name

# Check if target column exists
if target_column not in train_df.columns:
    raise KeyError(f"Target column '{target_column}' not found in training data.")
if target_column not in test_df.columns:
    raise KeyError(f"Target column '{target_column}' not found in test data.")

# Feature and target separation
features = [col for col in train_df.columns if col != target_column]

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(train_df[features])
y_train = train_df[target_column]

X_test = scaler.transform(test_df[features])
y_test = test_df[target_column]

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()